In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import os
import scipy.io as sio

from statsmodels.tsa.ar_model import AR

import tqdm
import  time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore') #忽略警告

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture

import xgboost

In [3]:
sample_rate = 256
origin_channel = 16


SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

CLIP_FORWARD = 1 #首部裁掉时间
CLIP_BACKWARD = 1 #尾部裁掉时间

trial_time = 3 #segment second


#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频

from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
def load_data(filename):

    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
   
    return data 

In [5]:
def separate(data , label , overlap_length):
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0
    
    while idx<=data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)
        
    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):    
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/0/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/1/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/2/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/3/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/4/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/5/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/6/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/7/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/8/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('../incremental/data/base_rf/%s/0/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('../incremental/data/base_rf/%s/1/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('../incremental/data/base_rf/%s/2/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('../incremental/data/base_rf/%s/3/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('../incremental/data/base_rf/%s/4/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('../incremental/data/base_rf/%s/5/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('../incremental/data/base_rf/%s/6/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('../incremental/data/base_rf/%s/7/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('../incremental/data/base_rf/%s/8/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
        
    return train_data , train_labels


In [7]:
def session_data_labels(session_id , freq , is_training):
    if is_training:
        overlap_length = 256*2
    else:
        overlap_length = 0
    
    str_freq = str(freq)
    
    subjcets = os.listdir('../incremental/data/incremental/%s/s%d/' % (str_freq , session_id)) #受试者ID
    
    data = []
    labels = []
    
    for subjcet in subjcets:
        filenames = os.listdir('../incremental/data/incremental/%s/s%d/%s/' % (str_freq , session_id , subjcet))
        
        person = np.concatenate([load_data('../incremental/data/incremental/%s/s%d/%s/%s' % (str_freq , session_id , subjcet , filename)) for filename in filenames] , axis = 0)
        
        person_data , person_label = separate( person , label = int(subjcet) , overlap_length = overlap_length)
        
        data.append(person_data)
        labels.append(person_label)
    
    #合并数据
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    
    #shuffle
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels


In [8]:
def get_center(data , label):
    centers = []
    
    for label_id in range(9): #一共9个受试者
        equal_idx = np.where(label == label_id)
    
        center = np.mean(data[equal_idx] , axis = 0)
        centers.append(center)
        
    return np.array(centers)

def get_center_simple(data):
    '''
    计算单个用户的脑电的中心
    '''
    return np.mean(data , axis=0)

In [9]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , LSTM

In [10]:
encoder_input = Input(shape = (256*3 , 16))

#encoder
encoder = LSTM(32 , return_sequences=True)(encoder_input)
encoder_output = Dense(64 , activation='relu')(encoder)
encoder_output = Flatten()(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(768*64 , activation='relu')(encoder_output)
decoder = Reshape(target_shape=(768 , 64))(decoder)
decoder = LSTM(32)(decoder)
decoder_output = Dense(256*3*16 , activation='relu')(decoder)
decoder_output = Reshape(target_shape=(256*3 , 16))(decoder_output)

In [11]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# PCA降维
## kmeans聚类

## 注意 分两次运行 6 7.5 和 8.5 10

In [13]:
for freq in [6 , 7.5]:
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  6
session :  1 [[-103.30456   -42.774944]
 [ 990.41473     6.174515]
 [-145.89462   493.09192 ]
 [-140.4229   -111.534676]
 [-139.74927   597.19586 ]
 [-147.90546   346.20505 ]
 [-150.65353    63.878716]
 [-156.25842   -23.859875]
 [-144.73189   -48.534504]] 0.096361
session :  3 [[-1.4558879e+02 -1.8367999e+00]
 [ 1.1654363e+03  2.5297263e-01]
 [-1.4851292e+02  4.1702633e+01]
 [-1.4292363e+02 -3.9863430e+01]
 [-1.4500789e+02 -1.0390284e+01]
 [-1.4408551e+02 -2.3640768e+01]
 [-1.4617097e+02  7.2626395e+00]
 [-1.4800337e+02  3.4528839e+01]
 [-1.4532034e+02 -5.9245410e+00]] 0.05057059999944613
session :  5 [[ 195.77013   -33.556423]
 [-101.32226   -24.268118]
 [  29.592646  140.68901 ]
 [ 167.79861    53.64836 ]
 [  38.217876   65.95049 ]
 [ 172.45627   -67.90003 ]
 [ -34.919937   97.60494 ]
 [ 247.20178    73.007256]
 [-108.37393    15.763445]] 0.09629039999890665
session :  6 [[-59.345722  -55.152966 ]
 [495.9683      5.7400436]
 [-47.148582  135.40564  ]
 [-92.9052    262.7744

ResourceExhaustedError: OOM when allocating tensor with shape[32,768,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: lstm_2/zeros_like = ZerosLike[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](reshape_1/Reshape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_25/mul/_3577 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3501_loss_25/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [12]:
for freq in [8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  8.5
session :  1 [[ -31.723572  -42.898712]
 [ 350.3053    155.9407  ]
 [-204.6263    269.1208  ]
 [   7.566886  -80.769295]
 [ -58.10243   -59.060368]
 [  35.45666   -97.25578 ]
 [ -30.031178  -64.00862 ]
 [-221.98976   308.12787 ]
 [  -4.66879   -13.238199]] 0.08240399999885994
session :  3 [[-63.422047  -72.85427  ]
 [342.97824    11.071364 ]
 [-71.45408   203.32275  ]
 [-30.139849  -26.565655 ]
 [-42.127884    3.7717018]
 [-38.698097  -45.06139  ]
 [382.1682     14.753518 ]
 [-30.86456    28.599566 ]
 [-40.086662  -31.166267 ]] 0.05486860000019078
session :  5 [[ 165.77715    139.71977  ]
 [   9.865319   -34.81572  ]
 [ -67.51116     10.893671 ]
 [  54.349102   -84.14891  ]
 [-145.87852     77.277824 ]
 [  41.775963   -30.38363  ]
 [ -23.136066    -3.6104004]
 [  79.18785    -68.01758  ]
 [  42.559483  -106.16223  ]] 0.056353200001467485
session :  6 [[ 313.23605    245.0128   ]
 [-131.49294     40.055923 ]
 [  87.609085  -147.69553  ]
 [ 175.73784   -168.79117  ]
 [-202.39

## DBSCAN聚类

In [11]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        dbscan = DBSCAN()
        start = time.clock()
        res = dbscan.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)

freq :  6
session :  1 [[-138.7129    297.2712  ]
 [1310.5975    -93.60619 ]
 [-250.63196  -272.39484 ]
 [-169.59291   -44.553272]
 [ -58.229546  186.34444 ]
 [-185.2595    478.83725 ]
 [  67.37501   263.4952  ]
 [-121.95158    78.429146]
 [-209.90662  -171.76633 ]] 0.059514700000000004
session :  3 [[-1.8803296e+02  8.9529400e+00]
 [ 1.5078998e+03 -2.0213323e+00]
 [-2.0117903e+02 -4.8392544e+01]
 [-1.6836658e+02  7.1393005e+01]
 [-1.7869119e+02  3.2342598e+01]
 [-1.9141826e+02 -7.3650522e+00]
 [-2.0328111e+02 -6.5394836e+01]
 [-1.8588985e+02  1.8041756e+01]
 [-1.8926936e+02  6.0011333e-01]] 0.04860399999999743
session :  5 [[-338.1623   -167.72446 ]
 [ 722.29926   -47.108364]
 [-250.8307    214.95482 ]
 [ -14.868722  175.67842 ]
 [-307.87552  -121.93938 ]
 [ 513.952      21.243168]
 [-178.19682   363.8366  ]
 [ 630.1675    -16.565285]
 [-276.96088   -66.96561 ]] 0.054015800000001946
session :  6 [[1956.3553     33.219997]
 [-238.5462   -152.66092 ]
 [-326.80298  1084.9767  ]
 [-388.49

session :  13 [[ -708.7093    -232.54137 ]
 [ 1688.3551   -1077.4382  ]
 [  925.8351    1491.0358  ]
 [ -215.08488    -48.08603 ]
 [   19.80037    314.6726  ]
 [ -406.39957   -117.592545]
 [ -134.5351     136.1686  ]
 [  101.81758    411.3189  ]
 [ -327.65445    -85.18546 ]] 0.053002599997853395
freq :  10
session :  1 [[-436.2955     60.680683]
 [1230.6445    821.02124 ]
 [ 912.3335   -850.08527 ]
 [  45.79504  -469.46237 ]
 [ 116.70336   225.19972 ]
 [-689.6668    115.36678 ]
 [-116.993355  -98.01127 ]
 [-388.26395   139.173   ]
 [ 109.4816   -585.2302  ]] 0.053358599994680844
session :  3 [[-623.27313  -318.4388  ]
 [3542.4705   -276.71674 ]
 [  24.309032  934.30554 ]
 [-483.742    -150.21236 ]
 [-652.4061   -387.99097 ]
 [-522.5905    -94.8728  ]
 [ -12.143808  973.49304 ]
 [ -17.647076  881.2186  ]
 [-570.4142   -204.91429 ]] 0.05186659999890253
session :  5 [[ 345.22232   748.5816  ]
 [-290.55136  -535.7936  ]
 [1169.4149   -657.02515 ]
 [-315.8828   -217.58473 ]
 [  10.775166  3

## GMM聚类

In [13]:
for freq in [6]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制

        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start

        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start

        _centers = get_center(X_pca , res)

        print('session : ' , session_id , _centers , time1 + time2)

session :  1 [[ -91.78708    -16.833883 ]
 [ 248.13307   -155.13446  ]
 [  51.363068   209.52211  ]
 [  29.44086     85.26891  ]
 [ 165.26932    241.40056  ]
 [ -35.35166    -30.56431  ]
 [ -67.79885    -24.08618  ]
 [  61.942184    71.33306  ]
 [  -7.6164813   57.3298   ]] 0.07418089999998756
session :  3 [[-54.74358   -23.332043 ]
 [390.25275    -1.8816473]
 [-47.62751     5.026705 ]
 [-43.50922    21.48915  ]
 [-50.665085   -7.3938265]
 [-39.697144   34.401028 ]
 [-57.384594  -34.021732 ]
 [-56.210957  -28.986357 ]
 [-48.70807     1.658791 ]] 0.16952269999995906
session :  5 [[-73.278336  -73.63143  ]
 [ -2.9997945  95.09365  ]
 [230.32646   -56.312935 ]
 [ 82.046295    1.8195336]
 [-59.363316  -46.19155  ]
 [ -7.2752237 119.64452  ]
 [-27.621841   66.1112   ]
 [ 38.232952  -29.541883 ]
 [-66.57222   -54.75765  ]] 0.023134400000003552
session :  6 [[-30.771193  -69.13502  ]
 [-74.95517   425.57748  ]
 [506.19232    11.572957 ]
 [-80.36419   -56.68371  ]
 [ -1.7830584 -70.70141  ]
 [

In [12]:
for freq in [7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  7.5
session :  1 [[ 653.37946  -148.87994 ]
 [-151.97038   369.84442 ]
 [-245.74887  -163.45732 ]
 [ 283.1422    -24.89151 ]
 [-481.425    -406.98865 ]
 [ 145.47415   215.82129 ]
 [ -61.25869    35.663277]
 [ 318.3756   -190.14497 ]
 [-248.6049     19.545584]] 0.025549099999999998
session :  3 [[1210.3219     50.227375]
 [-157.66637    64.510155]
 [ -79.94674  -356.3035  ]
 [-122.117615  391.6825  ]
 [-319.07422  -155.45422 ]
 [ 231.83759   -85.95908 ]
 [-389.58737   213.98291 ]
 [-249.26775  -286.95923 ]
 [-176.89607   500.66205 ]] 0.026399999999995316
session :  5 [[ 537.4364    320.09296 ]
 [-200.3565   -184.70947 ]
 [1209.6301   -287.48343 ]
 [-112.040634  546.48285 ]
 [ -67.833     184.95618 ]
 [-465.0962   -318.1079  ]
 [-337.08685   187.87296 ]
 [-418.76874   -19.725586]
 [-282.3576   -206.36183 ]] 0.01626009999999667
session :  6 [[ 257.26303  -183.34418 ]
 [-225.19684   127.019424]
 [1141.8859    353.35312 ]
 [  76.15762  -270.29596 ]
 [-146.72975    43.107773]
 [-301.

session :  13 [[ 129.08302   162.93819 ]
 [-473.76144   -45.749844]
 [1143.5576    -81.22331 ]
 [ 107.48623  -307.57227 ]
 [-364.10718   344.10098 ]
 [-518.4698   -298.47534 ]
 [ 170.15355   381.8895  ]
 [ 111.55904  -143.54237 ]
 [-383.26562   -15.512603]] 0.015172399998846231


# LDA降维
## kmeans聚类

In [12]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_lda)
        time2 = time.clock() - start
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  6
session :  1 [[-25.6246023   14.36935157]
 [137.75071244  11.03785453]
 [ -1.71922801 -43.45280592]
 [-16.4865442    7.57432151]
 [-21.59075801  13.90318967]
 [-25.96008755  10.01451854]
 [-20.12644163  15.42849445]
 [ -1.79417212 -40.23563339]
 [-24.17925322  13.10385384]] 0.6353644
session :  3 [[-8.13479664e+03 -4.93405742e+00]
 [ 6.50832425e+04 -2.11642321e-02]
 [-8.13521334e+03  2.53598069e+01]
 [-8.13565292e+03 -3.88221894e+01]
 [-8.13521675e+03 -2.86872136e+01]
 [-8.13492604e+03  2.06505972e+00]
 [ 6.50783809e+04  2.31456776e-02]
 [-8.13507773e+03  2.30824517e+01]
 [-8.13499904e+03 -7.98270308e-01]] 0.04940170000000066
session :  5 [[-19.14734916 -33.40981473]
 [155.85068195  -1.2562522 ]
 [-23.87039253  30.3098971 ]
 [-12.44415211  49.2598178 ]
 [ -8.82616428 -33.36600845]
 [-27.15335428 -37.87660799]
 [-27.70415589  25.48823809]
 [-14.83397998  41.85019453]
 [-21.94945335 -41.08746737]] 0.0687329999999946
session :  6 [[ -317.1438277    101.12463226]
 [ 1605.98925373

session :  9 [[-226.14707825  612.2313028 ]
 [-138.35624305  -82.94055009]
 [1138.2878787    46.88380107]
 [-110.81786458 -136.90171387]
 [-121.64804777  -93.63998856]
 [-114.61152553  -91.66801723]
 [-142.08205481  -86.11090129]
 [-142.87620384  -79.0186892 ]
 [-225.5900008   607.23205388]] 0.050406799997290364
session :  11 [[  30.92153207  -71.23997873]
 [-434.65121491  165.36565435]
 [ 263.53972455  314.20083313]
 [  11.44910711  -63.22726917]
 [  16.89862607  -73.02748143]
 [  38.46245649  -71.78700897]
 [  15.00861469  -61.67904062]
 [  27.16819332  -69.88417564]
 [ 266.08133504  318.89832444]] 0.050331499998719664
session :  12 [[ 147.3564638   212.45163636]
 [ -91.52060514  -78.62968021]
 [ 874.0514238   -42.66242975]
 [-202.54474979   66.45263443]
 [-255.05217759   -6.22825379]
 [-183.4732338    49.10947729]
 [ -77.95427458  -53.39105671]
 [-111.29976972  -73.28771162]
 [ -99.56307699  -73.81461599]] 0.050614499999937834
session :  13 [[  69.28280362   -3.57437942]
 [-422.4194

## DBSCAN聚类

In [12]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        dbscan = DBSCAN()
        start = time.clock()
        res = dbscan.fit_predict(X_lda)
        time2 = time.clock() - start
        
        _centers = get_center(X_lda , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  6
session :  1 [[-3.22850110e+02  6.12595230e+00]
 [ 2.58308942e+03  1.07927494e-02]
 [-3.23069478e+02 -7.84192095e+01]
 [-3.22982370e+02  3.41671816e+01]
 [-3.22719874e+02  1.17250755e+01]
 [-3.22860318e+02  3.69859546e+00]
 [-3.22868678e+02  7.57162030e+00]
 [ 2.58624626e+03 -3.25814993e-02]
 [ 2.57966675e+03  7.26451519e-03]] 0.10900409999999999
session :  3 [[-3.53367217e+04 -3.73520024e-01]
 [ 2.82691911e+05  5.38366617e-04]
 [-3.53367059e+04  4.64452019e+01]
 [-3.53367242e+04 -1.77473339e+01]
 [-3.53367168e+04  2.17672388e+01]
 [-3.53367244e+04 -1.23320259e+01]
 [-3.53367290e+04 -4.34638475e+00]
 [-3.53367276e+04 -2.26718947e+01]
 [ 2.82696753e+05 -8.72408905e-04]] 0.057765899999999704
session :  5 [[-29.90827387  -6.55877904]
 [172.95723982 -19.89024827]
 [ 18.66989258  76.65201441]
 [ -9.14645592   7.67935668]
 [-31.84400806 -12.30673395]
 [-22.65979469  -1.51058541]
 [-36.81496625 -17.90385772]
 [-29.27467987 -13.88483956]
 [-20.52048139  16.92369052]] 0.05323609999999

session :  9 [[  -3.22241793  325.73038751]
 [1268.771007   -559.05484215]
 [-681.9302497  -682.48986122]
 [-484.74320431 -532.07947455]
 [-105.47488136  143.23694469]
 [  12.06970356  328.00927337]
 [  -5.4030545   335.83931077]
 [  -2.11673395  316.86581126]
 [   2.04983119  323.94245032]] 0.05064429999765707
session :  11 [[  78.07759166  -48.3742535 ]
 [  54.94697052  427.5032315 ]
 [-630.87140264   11.83992561]
 [ -13.47903998 -161.15766727]
 [ 166.33912644   53.85643879]
 [  78.09245359 -107.85573341]
 [  68.4690692   -77.26774008]
 [ 106.13327356  -40.91150068]
 [  92.29195766  -57.63270097]] 0.04918240000188234
session :  12 [[-199.42291135 -370.74849375]
 [1522.99385503   -2.13418367]
 [-191.25380329   53.02037104]
 [-179.53763359   83.22285371]
 [-202.63169663   73.06592202]
 [-181.3548683    34.30376814]
 [-188.38929038   45.71074355]
 [-192.24066884   38.76469773]
 [-198.22173965 -375.61258144]] 0.05075329999817768
session :  13 [[-157.98060333 -100.41572469]
 [1463.6353656

## GMM聚类

In [12]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_lda)
        time2 = time.clock() - start
        
        _centers = get_center(X_lda , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  6
session :  1 [[-1.05944182e+02  5.76240059e+00]
 [ 8.48244779e+02 -1.37549974e-01]
 [-1.07700932e+02 -4.49329347e+01]
 [-1.05913446e+02  6.46644719e+00]
 [ 8.52742345e+02  1.60335730e-01]
 [-1.08174295e+02 -4.24707558e+01]
 [ 8.40974269e+02 -7.16269125e-01]
 [-1.07410935e+02 -4.74683308e+01]
 [ 8.51372179e+02 -4.63609233e-02]] 0.09038009999999999
session :  3 [[-1.80740271e+04  2.36411949e+00]
 [ 1.44594975e+05  5.38491764e-03]
 [-1.80739619e+04 -3.26375852e+01]
 [-1.80740402e+04  1.37983117e+01]
 [-1.80739919e+04 -1.64978005e+01]
 [-1.80740375e+04  5.80255776e+00]
 [ 1.44590200e+05 -3.75535977e-03]
 [-1.80740345e+04  9.25635226e+00]
 [-1.80740429e+04  1.07599820e+01]] 0.02790470000000056
session :  5 [[-139.52698914  -48.13589154]
 [  80.8080502     2.10092703]
 [-201.34419649   66.29071937]
 [-150.55734895  -33.10724701]
 [  85.87565837    4.82093042]
 [-198.17571812   62.15849981]
 [  82.28949585    2.8879785 ]
 [-143.10905867  -46.04037363]
 [-201.05902059   63.74071812]]

InternalError: Blas GEMM launch failed : a.shape=(32, 16), b.shape=(32, 32), m=16, n=32, k=32
	 [[Node: training_1/Adam/gradients/lstm_1/while/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/lstm_1/while/MatMul_3_grad/MatMul_1/StackPopV2, training_1/Adam/gradients/lstm_1/while/add_1_grad/Reshape)]]

In [12]:
for freq in [7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = (data-np.mean(data)) / np.std(data)

        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_lda)
        time2 = time.clock() - start
        
        _centers = get_center(X_lda , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  7.5
session :  1 [[ 100.12995683  -11.50707727]
 [-220.97614688  -89.04497864]
 [-150.58792913  306.3803245 ]
 [-247.39387004 -139.03134687]
 [ 103.54992611  -14.74612217]
 [ 102.1659803   -13.0144072 ]
 [ 105.74561361  -12.75764762]
 [-224.3162141   -89.82464991]
 [-221.15126852  -89.24772133]] 0.0975809
session :  3 [[ -15.93191664   35.16101751]
 [ 232.93882354  -23.05476094]
 [ -54.75138712 -178.52700447]
 [ -51.09923002   15.05518163]
 [ -34.62467386   22.7977079 ]
 [ -17.63035554   30.02786003]
 [ -18.16680354   36.70670122]
 [ -52.61132979   10.42809691]
 [ -20.84171735   29.13564353]] 0.034459800000000484
session :  5 [[ -31.32265981  -43.33804382]
 [ 320.26383428   75.18109513]
 [-131.96752861  254.19760207]
 [  10.7428909   -44.65181344]
 [ -30.76390978  -52.0472465 ]
 [ -37.33738172  -38.1845188 ]
 [ -37.32487353  -56.60596668]
 [ 324.5283548    76.6328698 ]
 [ -33.67577375  -43.22895833]] 0.02974769999999438
session :  6 [[ -47.03842346  184.64390806]
 [1005.3893963

session :  8 [[ -87.6118061   -50.82862769]
 [ 752.1239358    10.37780937]
 [-120.97749329  256.56063402]
 [-102.05205442    9.04087108]
 [ -94.15291027  -20.16738408]
 [ -88.63125569  -58.60672059]
 [ -88.03876189  -43.46379893]
 [ -86.2788584   -51.96108246]
 [ -84.42118007  -50.97317692]] 0.05598700000336976
session :  9 [[ -36.28221529   37.36656343]
 [ 392.65006043  -41.19760589]
 [-123.26759109 -233.25014669]
 [  14.87804094   13.88434404]
 [ -76.57031808   57.86066159]
 [ -42.08835605   62.55653387]
 [ -50.26747692   30.07010891]
 [ -41.35264219   30.32965936]
 [ -37.69950175   42.37988138]] 0.022577600000659004
session :  11 [[-65.70404913 -29.64592946]
 [541.71434202   5.23557549]
 [-72.89686062 131.7859398 ]
 [-84.22968757  85.74197152]
 [-57.60647258 -59.18630728]
 [-67.80824875  -7.77336667]
 [-65.78916937 -42.13585913]
 [-64.58760711 -23.35395848]
 [-63.09224687 -60.66806579]] 0.02314710000064224
session :  12 [[ 961.38672683   -3.49381343]
 [-123.48917005  -35.42806771]
 